<h1>Convolutional neural network for simultaneous prediction of several soil properties using infrared spectra - Tutorial
<span class="tocSkip"></span></h1>
By Wartini Ng
&#169; Sydney Institute of Agriculture, 2021 

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-Libraries" data-toc-modified-id="Import-Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Libraries</a></span></li><li><span><a href="#Data-preparation" data-toc-modified-id="Data-preparation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data preparation</a></span><ul class="toc-item"><li><span><a href="#Load-data" data-toc-modified-id="Load-data-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Load data</a></span><ul class="toc-item"><li><span><a href="#Extract-the-wet-chemistry-data" data-toc-modified-id="Extract-the-wet-chemistry-data-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Extract the wet chemistry data</a></span></li><li><span><a href="#Extract-the-spectra-data" data-toc-modified-id="Extract-the-spectra-data-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Extract the spectra data</a></span></li></ul></li><li><span><a href="#Exploratory-data-analysis" data-toc-modified-id="Exploratory-data-analysis-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Exploratory data analysis</a></span><ul class="toc-item"><li><span><a href="#Lab-measurements" data-toc-modified-id="Lab-measurements-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Lab measurements</a></span></li><li><span><a href="#Spectral-data" data-toc-modified-id="Spectral-data-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Spectral data</a></span></li></ul></li></ul></li><li><span><a href="#Accuracy-evaluation" data-toc-modified-id="Accuracy-evaluation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Accuracy evaluation</a></span></li><li><span><a href="#Train-the-model" data-toc-modified-id="Train-the-model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Train the model</a></span><ul class="toc-item"><li><span><a href="#Method1:-Partial-Least-Square-Regression-(PLSR)" data-toc-modified-id="Method1:-Partial-Least-Square-Regression-(PLSR)-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Method1: Partial Least Square Regression (PLSR)</a></span></li><li><span><a href="#Method-2:-Convolutional-Neural-Network-(CNN)" data-toc-modified-id="Method-2:-Convolutional-Neural-Network-(CNN)-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Method 2: Convolutional Neural Network (CNN)</a></span><ul class="toc-item"><li><span><a href="#Split-the-train-data" data-toc-modified-id="Split-the-train-data-4.2.1"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>Split the train data</a></span></li><li><span><a href="#Spectral-data-standardization" data-toc-modified-id="Spectral-data-standardization-4.2.2"><span class="toc-item-num">4.2.2&nbsp;&nbsp;</span>Spectral data standardization</a></span></li><li><span><a href="#Reshape-data" data-toc-modified-id="Reshape-data-4.2.3"><span class="toc-item-num">4.2.3&nbsp;&nbsp;</span>Reshape data</a></span></li><li><span><a href="#Define-the-CNN-model" data-toc-modified-id="Define-the-CNN-model-4.2.4"><span class="toc-item-num">4.2.4&nbsp;&nbsp;</span>Define the CNN model</a></span><ul class="toc-item"><li><span><a href="#Single-output-predictions" data-toc-modified-id="Single-output-predictions-4.2.4.1"><span class="toc-item-num">4.2.4.1&nbsp;&nbsp;</span>Single output predictions</a></span></li><li><span><a href="#Multiple-output-predictions" data-toc-modified-id="Multiple-output-predictions-4.2.4.2"><span class="toc-item-num">4.2.4.2&nbsp;&nbsp;</span>Multiple output predictions</a></span></li></ul></li><li><span><a href="#Visualize-the-feature-maps-in-the-convolution-layers" data-toc-modified-id="Visualize-the-feature-maps-in-the-convolution-layers-4.2.5"><span class="toc-item-num">4.2.5&nbsp;&nbsp;</span>Visualize the feature maps in the convolution layers</a></span></li><li><span><a href="#Variables-of-Importance" data-toc-modified-id="Variables-of-Importance-4.2.6"><span class="toc-item-num">4.2.6&nbsp;&nbsp;</span>Variables of Importance</a></span></li></ul></li></ul></li></ul></div>

This tutorial demonstrates the application of [Convolutional Neural Network (CNN)](https://developers.google.com/machine-learning/glossary/#convolutional_neural_network) for the prediction of soil properties using infrared spectroscopy. In this tutorial, we focus on developing the application of 1-Dimensional CNN (CNN1D) as a regression model.

The data used in this notebook is a subset of the actual dataset used in the [original study](https://doi.org/10.1016/j.geoderma.2019.06.016). Hence, the architecture of the CNN model is also adjusted accordingly. Details of the original study can be found: <br>

Ng, W., Minasny, B., Montazerolghaem, M., Padarian, J., Ferguson, R., Bailey, S. and McBratney, A.B., 2019. Convolutional neural network for simultaneous prediction of several soil properties using visible/near-infrared, mid-infrared, and their combined spectra. Geoderma, 352, pp.251-267. https://doi.org/10.1016/j.geoderma.2019.06.016

The code is written by Wartini Ng &#169; Sydney Institute of Agriculture 2021.

In [ ]:
import time
start = time.time()

# Import Libraries

First we need to load some Python libraries that will be used

In [ ]:
## load library
from __future__ import print_function, division
import scipy.io as sio
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from keras import models, layers, initializers
from keras.layers import Input, Conv1D,  Dense,  MaxPooling1D,  Flatten, Activation,  Dropout, GaussianNoise, Reshape, BatchNormalization, Convolution2D,  MaxPooling2D
from keras.models import Sequential, Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau,  TensorBoard
from keras.wrappers.scikit_learn import KerasRegressor
from keras.utils import np_utils

from scipy import stats
from scipy.signal import spectrogram
from scipy.stats import iqr
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from datetime import datetime

# Data preparation
## Load data

The dataset used in the notedbook can be downloaded from [google drive](https://drive.google.com/file/d/1gZD9VG3bAd0PqGmrJ_80flyEwNh1sATk/view?usp=sharing).

The dataset has been compressed in the `.npz` file format. This file contains list of arrays for various variables. This dataset contains a total of 1000 samples which has been split into training (80%) and test set (20%). These samples have been analyzed for organic carbon, cation exchange capacity, clay and sand content. The mid-infrared (MIR) spectra of the samples were collected using Vertex 70 (BrukerOptics, Ettlingen, Germany), which covered a spectral range
between 7498 and 600 $cm^{-1}$ with a resolution of 4 $cm^{-1}$.

The data can be downloaded directly from the notebook with the command:

In [ ]:
import gdown
!gdown --id 1gZD9VG3bAd0PqGmrJ_80flyEwNh1sATk

If you run it within the google colab environment, the data has been downloaded into your google drive and can be retrieved as:

In [ ]:
Data = np.load('/content/data.npz')

# To view what objects are available within the npz file
Data.files

### Extract the wet chemistry data 

In [ ]:
y_train = Data['y_train']
y_test = Data['y_test']

print("Training set: {}".format(y_train.shape))  # 800 samples with 4 properties
print("Testing set:  {}".format(y_test.shape)) # 200 samples with 4 properties

There are **four** different measurements of soil properties:
1. Organic Carbon
2. CEC
3. Clay
4. Sand

Each of the soil properties was measured in a different scale.

### Extract the spectra data

In [ ]:
X_train_MIRraw = pd.DataFrame(Data['MIRtrain'])
X_test_MIRraw = pd.DataFrame(Data['MIRtest'])

# name the columns for the spectra data, as we need this information later to trim data from certain wavelengths
wave = np.array(Data['wave']).reshape(-1,)
X_train_MIRraw.columns = wave
X_test_MIRraw.columns = wave

Check to ensure that we have same spectra dimensions for train and test dataset.

In [ ]:
print("Training set: {}".format(X_train_MIRraw.shape))  # 800 samples with 3578 wavelengths
print("Testing set:  {}".format(X_test_MIRraw.shape)) # 200 samples with 3578 wavelengths

## Exploratory data analysis

### Lab measurements

In [ ]:
# Each of the properties have slightly different ranges.
properties = ['OC','CEC','Clay','Sand']

#Check the spread of the train data
pd.DataFrame(y_train, columns = properties).describe()

**Note that the first column in Python starts at zero, instead of one.**

In [ ]:
x=0 # this indicates the data of column 0
plt.hist(y_train[:,x])

In [ ]:
#Check the spread of the test data
pd.DataFrame(y_test, columns = properties).describe()

<h2>Check for null and missing values<span class="tocSkip"></span></h2>

In [ ]:
pd.DataFrame(y_train, columns = properties).isnull().any()

In [ ]:
pd.DataFrame(y_test, columns = properties).isnull().any()

Seems like there is no missing values in both train and the test dataset.

<h2>Correlations<span class="tocSkip"></span></h2>


In [ ]:
# determine the correlations 
corr = pd.DataFrame(y_train, columns = properties).corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True

# derive the correlation plot
sns.heatmap(corr, mask=mask, annot=True, cmap=sns.diverging_palette(20, 220, n=200))

There is a strong correlation betweeb Clay and CEC, which is expected. A strong negative correlation of sand and clay content were also observed.

### Spectral data
Plot the original spectral data:

In [ ]:
ticks_font = 10
label_font = 15

plt.figure(figsize=(10,3))

plt.subplot(1, 2, 1)
plt.plot(X_train_MIRraw.T, color='lightgrey');
plt.gca().invert_xaxis()
plt.xlabel('Wavenumber ($cm^{-1}$)', fontsize=label_font)
plt.xticks(fontsize=ticks_font)
plt.yticks(fontsize=ticks_font)
plt.title('Train samples')

plt.subplot(1, 2, 2)
plt.plot(X_test_MIRraw.T, color='lightgrey') 
plt.gca().invert_xaxis()
plt.xlabel('Wavenumber ($cm^{-1}$)', fontsize=label_font)
plt.yticks(fontsize=ticks_font)
plt.title('Test samples');

Notice that the wavelengths for the MIR data ranges from ~ 7500 - 600 $cm^{-1}$. Typically, the wavelength range utilized in the MIR is between 4000 - 700 $cm^{-1}$. Hence, the spectra will be trimmed to that range.

In this notebook, the we applied several spectral pre-processing method to the spectral data. The aim of the pre-processing method is to improve the quality of the spectra before being implemented in the modeling. 

The pre-processing method applied in this notebook are:
1. Spectra trimming and resampling
2. Savitzky-Golay filtering: noise removal
3. Standard normal variate transformation (SNV): scatter correction

Note that there is no universal best method of pre-processing. 
**Trial and error** based on your data is recommended.

Then, we develop some functions to do the spectra pre-processing mentioned earlier:

In [ ]:
# Pre-processing functions
from scipy import interpolate
from scipy.signal import savgol_filter
import numpy as np

def filter_spectra(spectra, method, w = 11, p = 2, m = 0, **kwargs):
    """Define functions for spectra pretreatment based on various methods
    
    Parameters
    ----------
    spectra : ndarray
        Array of spectral matrix
    method : str
        'S-Golay': Savitzky Golay smoothing
        'SNV': Standard normal variate transformation
        'resample': resample the wavelength
        
    kwargs
    ------
    w : length of the filter window (i.e., the number of coefficients).
        `window_length` must be a positive odd integer.
    p : polynomial_order
        The degree of the polynomial (0 for a constant), default = 2
    m : derivative
        The degree of derivative used within the S-Golay method, default = 0
        
    Outputs
    -------
    out : ndarray
        Contain the corrected spectra for a specific method
    """
    
    [M, N] = spectra.shape
    
    if method == 'S-Golay':
        treated_spec = savgol_filter(spectra, window_length = w, polyorder = p, deriv = m)
    elif method == 'SNV':
        treated_spec = np.zeros([M, N])
        for i in range(0, M):
            temp = spectra[i,:]
            treated_spec[i,:]=(temp - np.mean(temp))/np.std(temp)
    elif method == 'resample': 
        wave = spectra.columns.values.astype(float)
        new_wave = np.arange(3996,700,-4)
        interp_func = interpolate.interp1d(wave, spectra)
        treated_spec = interp_func(new_wave)

    else:
        raise ValueError("Method not found..")
    return treated_spec

def spectra_preprocessing (spectra):
    """ Compile all the spectral pre-processing compiled together
    
    Parameters
    ----------
    spectra : ndarray
        Array of spectral matrix
    
    Outputs
    -------
    out : ndarray
        Contain the corrected spectra that underwent various spectra pre-processing:
        1. resample and trim to a certain wavelength
        2. Smoothing through Savitzky Golay 
        3. Standard normal variate transformation
    
    """
    spec_processed = filter_spectra (filter_spectra (filter_spectra (spectra, 'resample'), 'S-Golay', m = 0), 'SNV')
    return spec_processed

In [ ]:
# derived the pre-processed spectra
wavet = np.arange(3996,700,-4)
X_train_MIRt = spectra_preprocessing (X_train_MIRraw)
X_test_MIRt = spectra_preprocessing (X_test_MIRraw)

In [ ]:
print("Train set: {}".format(X_train_MIRt.shape))  # 800 samples with 824 wavelengths
print("Test set:  {}".format(X_test_MIRt.shape)) # 200 samples with 824 wavelengths

In [ ]:
# plot the spectra after pre-treatment
plt.figure(figsize=(10,3))

plt.subplot(1, 2, 1)
plt.plot(wavet, X_train_MIRt.T, color='lightgrey');
plt.gca().invert_xaxis()
plt.xlabel('Wavenumber ($cm^{-1}$)', fontsize=label_font)
plt.xticks(fontsize=ticks_font)
plt.yticks(fontsize=ticks_font)
plt.title('Train samples')

plt.subplot(1, 2, 2)
plt.plot(wavet, X_test_MIRt.T, color='lightgrey') 
plt.gca().invert_xaxis()
plt.xlabel('Wavenumber ($cm^{-1}$)', fontsize=label_font)
plt.yticks(fontsize=ticks_font)
plt.title('Test samples');

We can observe that the spectra has been trimmed to the desired wavelengths, and the peaks from the same wavelengths can be compared better after the pre-processing.

# Accuracy evaluation

To assess how well a particular model in providing the predictions, we also write some code to determine the goodness of fit of the predictions.

In [ ]:
# Define functions to evaluate accuracy measures
def plot_goof(Yobs, Ypred):
    plt.scatter(Yobs, Ypred)
    plt.ylabel('Predicted Values', fontsize=13)
    plt.xlabel('Observed Values', fontsize=13)
    plt.xlim([np.min(Yobs), np.max(Yobs)])
    plt.ylim([np.min(Yobs), np.max(Yobs)])
    xpoints = ypoints = plt.xlim()
    plt.plot(xpoints, ypoints, linestyle='--', color='red', lw=2, scalex=False, scaley=False)

def goof(observed, predicted):
    cormat = np.corrcoef(observed, predicted)
    corr_xy = cormat[0, 1]
    r2 = corr_xy ** 2
    RMSE = np.sqrt(np.mean((observed - predicted) ** 2))
    bias = np.mean(predicted) - np.mean(observed)
    mx = np.mean(observed)
    my = np.mean(predicted)
    s2x = np.cov(observed)
    s2y = np.cov(predicted)
    sxy = np.mean((observed - mx) * (predicted - my))
    ccc = 2 * sxy / (s2x + s2y + (mx - my) ** 2)
    RPIQ = iqr(observed) / RMSE
    return r2, RMSE, bias, RPIQ, ccc

# Train the model
## Method1: Partial Least Square Regression (PLSR)

PLSR is a technique that attempts to combine Principal Component Analysis and multiple regression (Wold et al., 2001). It aims to predict a set of dependent variables (soil properties) by extracting from the spectra a set of ‘orthogonal’ factors (or so called latent variables) which give the best prediction. The components in partial least squares are determined by not only by the predictor variables but also by the response variable(s).

`sklearn` library already has a PLSR function, so we just need to call the function and use it without reinventing the wheel. We need to define the number of components we want to use in our PLS regression model. A 10-fold cross validation is then utilized to find the optimized number of components.

In [ ]:
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score

def optimise_pls_cv(X, y, n_comp):
    # Define PLS object
    pls = PLSRegression(n_components=n_comp)

    # Cross-validation
    y_cv = cross_val_predict(pls, X, y, cv=10)

    # Calculate scores
    r2 = r2_score(y, y_cv)
    mse = mean_squared_error(y, y_cv)
    
    return (y_cv, r2, mse)

In [ ]:
# test with 30 components
r2s = []
mses = []
rpds = []

ncomps = np.arange(1, 31)
for ncomp in ncomps:
    y_cv, r2, mse = optimise_pls_cv(X_train_MIRt, y_train[:,x], ncomp)
    r2s.append(r2)
    mses.append(mse)

In [ ]:
# select the optimum number of components
nc_opt = np.argmin(mses)
print('The optimum number of components is', ncomps[nc_opt])

In [ ]:
# retrain the model with the optimized number of components
pls = PLSRegression(n_components = ncomps[nc_opt])
pls.fit(X_train_MIRt, y_train[:,x])

# Predict on the test spectra
pls_vpred = pls.predict(X_test_MIRt)

In [ ]:
#Observe the fit of the data
plot_goof(y_test[:,x],pls_vpred)

In [ ]:
# Obtain the goodness of fit measures
r2, RMSE, bias, RPIQ, _ = goof(y_test[:,x], np.reshape(pls_vpred,-1))
print('PLSR - R2: %0.3f, RMSE: %0.3f, bias: %0.3f, RPIQ:%0.3f' %(r2, RMSE, bias, RPIQ))

## Method 2: Convolutional Neural Network (CNN)

The following are the codes for fitting a 1D CNN model to the spectra data to predict soil properties. 

### Split the train data
A portion of the train dataset is taken out to monitor and tune the model performance. In this case, the dataset is split into 90% calibration and 10% validation. The validation set can be used to adjust hyperparameter values.

In [ ]:
# Set the random seed
random_seed = 2021

In [ ]:
# Split the train dataset into calibration and validation (used to fine tune the model).
X_cal_MIR, X_val_MIR, y_calX, y_valX = train_test_split(X_train_MIRraw, y_train, train_size=0.9, random_state=random_seed)
X_cal_MIR.shape, X_val_MIR.shape, y_calX.shape, y_valX.shape

### Spectral data standardization
The spectra standardization is implemented using spectral trimming and resampling (to ensure the input are comparable), followed by SNV transformation.

In [ ]:
X_cal_MIR = filter_spectra (filter_spectra (X_cal_MIR, 'resample'), 'SNV')
X_val_MIR = filter_spectra (filter_spectra (X_val_MIR, 'resample'), 'SNV')
X_test_MIR = filter_spectra (filter_spectra (X_test_MIRraw, 'resample'), 'SNV')

# plot the spectra after pre-treatment
plt.figure(figsize=(10,3))
ticks_font= 15
plt.subplot(1, 3, 1)
plt.plot(wavet, X_cal_MIR.T, color='lightgrey');
plt.gca().invert_xaxis()
plt.xlabel('Wavenumber ($cm^{-1}$)', fontsize=label_font)
plt.title('Calibration samples')

plt.subplot(1,3, 2)
plt.plot(wavet, X_val_MIR.T, color='lightgrey') 
plt.gca().invert_xaxis()
plt.xlabel('Wavenumber ($cm^{-1}$)', fontsize=label_font)
plt.title('Validation samples');

plt.subplot(1,3, 3)
plt.plot(wavet, X_test_MIR.T, color='lightgrey') 
plt.gca().invert_xaxis()
plt.xlabel('Wavenumber ($cm^{-1}$)', fontsize=label_font)
plt.title('Test samples');

### Reshape data

To feed the data into the one-dimensional CNN model, the input data needs to be converted into 3D matrices. It requires that you specify the expected shape of the input images in terms of rows (height), columns (width), and channels (depth) or [rows, columns, channels]. So, we need to add an extra dimension in the end which correspond to number of channels.

In this example, the original spectra of 3578 wavelengths have been reduced to 824 to allow for faster computation. The number of channel  is one.

In [ ]:
# reshape the spectra to match input for the model
cal_spec = np.expand_dims(X_cal_MIR, axis=2)
val_spec = np.expand_dims(X_val_MIR, axis=2)
test_spec = np.expand_dims(X_test_MIR, axis=2)

nb_features = cal_spec.shape[1]; channel_number=1;

In [ ]:
print("Cal set: {}".format(cal_spec.shape))  # 600 samples with 824 wavelengths
print("Val set:  {}".format(val_spec.shape)) # 200 samples with 824 wavelengths
print("Test set:  {}".format(test_spec.shape)) # 200 samples with 824 wavelengths

### Define the CNN model

The first layer of the CNN model is the **convolutional layer (Conv1D)**. It is like a set of learnable filters. I choose to set 10 **filters** for the first layer and increase it further down the deeper layers. Each filter transforms a part of the spectral data (defined by the **kernel size**) using the kernel filter. The kernel filter matrix is applied on the whole spectral data. Filters can be seen as a transformation of the spectral data. The CNN can isolate features that are useful from these transformed spectral data (feature maps). When filter of certain size is applied into spectra data input, a reduction of  the resulting output feature map is observed, which is often referred as border effects. To fix the border effect problem, **padding** is applied to ensure that the output has the same shape as the input. Different sized filters will detect different sized features in the input image and, in turn, will result in differently sized feature maps. The filter is moved across the spectra data, and the amount of movements between applications of the filter to the input spectra data is referred to as **stride**. This can be used to downsample data. 

Rectifier linear activation (**relu**) is a piecewise linear function that will output the input directly if it is positive, otherwise, it will output zero. The relu function is used to add non linearity to the network.

The second important layer in CNN is the **pooling layer (MaxPooling1D)**. This layer simply acts as a downsampling filter. We implemented the maxpooling, which looks at the neighboring pixels and picks the maximal value. These are used to reduce computational cost, and to some extent also reduce overfitting. We have to choose the pooling size (i.e the area size pooled each time) more the pooling dimension is high, more the downsampling is important.

<!-- **Dropout** is a regularization method, where a proportion of nodes in the layer are randomly ignored (setting their wieghts to zero) for each training sample. This drops randomly a proportion of the network and forces the network to learn features in a distributed way. This technique also improves generalization and reduces the overfitting. -->

The **flatten layer (Flatten)** is use to convert the final feature maps into a one single 1D vector. This flattening step is needed so that you can make use of fully connected layers after some convolutional/maxpool layers. It combines all the found local features of the previous layers.

At the end, I used the features in two fully-connected (**Dense**) layers which is just artificial an neural networks (ANN) regression. In the last dense layer `(x = Dense (1, name='Dense_all.2') (x))`, the model generates the final prediction.

Once our layers are added to the model, we need to set up a loss function and an optimisation algorithm. The metric function "mae" is used is to evaluate the performance our model. This metric function is similar to the loss function, except that the results from the metric evaluation are not used when training the model (only for evaluation). Now that we have a cost measure that must be minimized, we can then create an optimizer. In this case it is the `AdamOptimizer` which is an advanced form of Gradient Descent.

Note, that all the layers need to be tweaked as needed. Further details of the layers can be found in [here](https://keras.io/api/layers/).

In [ ]:
# develop the model
def model1D(channel_number, nb_features, multOUT):
    
    inputShape = (nb_features,1)
    inputs = Input(shape=inputShape)
    f_s = 6  #kernel_size
    
    x = Conv1D(10, f_s, padding="same", strides=2, name="Conv1", activation ="relu")(inputs)
    x = MaxPooling1D(2) (x)

    x = Conv1D(20, f_s, padding="same", name="Conv2", activation ="relu")(x)
    x = MaxPooling1D(2) (x)

    x = Conv1D(20, f_s, padding="same",name="Conv3", activation ="relu")(x)
    x = MaxPooling1D(2) (x)

    x = Flatten () (x)

   #for multiple output predictions
    if (multOUT==True):
        d_units = 10
        
        y1 = Dense (d_units, activation='relu',name='Dense_OC.1') (x)
        y1 = Dense (1, name='Dense_OC.2') (y1)

        y2 = Dense (d_units, activation='relu',name='Dense_CEC.1') (x)
        y2 = Dense (1, name='Dense_CEC.2') (y2)

        y3 = Dense (d_units, activation='relu',name='Dense_clay.1') (x)
        y3 = Dense (1, name='Dense_clay.2') (y3)

        y4 = Dense (d_units, activation='relu',name='Dense_sand.1') (x)
        y4 = Dense (1, name='Dense_sand.2') (y4)

        model = Model(inputs=inputs,outputs=[y1,y2,y3,y4])

    #for single output prediction
    elif (multOUT == False):
        x = Dense (8, activation='relu', name='Dense_all.1') (x)
        x = Dense (1, name='Dense_all.2') (x)

        model = Model (inputs=inputs, outputs=x)

    #compile the model
    model.compile(loss='mse', optimizer=Adam(learning_rate=0.0005), metrics=['mae'])
    return model


**Define training parameters **
The number of `epochs` defines the number times that the learning algorithm will work through the entire training dataset. The `batch_size` is a hyperparameter that defines the number of samples to work through before updating the internal model parameters.

In [ ]:
# define other model parameters needed
EPOCH = 200      #epochs
B_size = 8       #batch_size

#### Single output predictions

In [ ]:
# In this example, we predict the first variable (OC)
# First we need to standardize the output prediction, just like when the spectra is standardized
sc_y = StandardScaler()
y_cal = sc_y.fit_transform(y_calX[:,x].reshape(-1,1))
y_val = sc_y.transform(y_valX[:,x].reshape(-1, 1))

In [ ]:
# create the model
# we are trying to predict one output at a time, so the multOUT options were set to False
model = model1D(channel_number, nb_features, multOUT=False)

#Let's display the architecture of your model so far:
model.summary()

Implement [keras callback API to reduce learning rate](https://keras.io/api/callbacks/reduce_lr_on_plateau/), when a metric has stopped improving.

In [ ]:
redLR = ReduceLROnPlateau(monitor='val_loss', factor=0.4, patience=30, verbose=1,  mode='min')

In [ ]:
# Train the model
h = model.fit(cal_spec, y_cal, validation_data=(val_spec,y_val), epochs=EPOCH, batch_size = B_size, callbacks=[redLR])

In [ ]:
# Evaluate the model by plotting the mean absolute error of cal and val data
plt.plot(h.history['mae'], label='mae')
plt.plot(h.history['val_mae'], label = 'val_mae')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend(loc='upper right')

The plot shows the error as a function of Epoch with training data and internal validation data. Ideally we tune the hyperparameters so we can reduce the validation errors. From the plot above, we can see that the mae val set became stagnant as the epochs increases.

In [ ]:
# create predictions using trained CNN model
tmp = model.predict(test_spec)

In [ ]:
#inverse back the results
Y_predT = sc_y.inverse_transform(tmp)

In [ ]:
# Plot the goodness of fit
plot_goof(y_test[:,x],Y_predT)

#  Obtain the goodness of fit metrics
r2, RMSE, bias, RPIQ ,_ = goof(y_test[:,x],Y_predT.reshape(-1))
print('CNN model - R2: %0.3f, RMSE: %0.3f, bias: %0.3f, RPIQ:%0.3f' %(r2, RMSE, bias, RPIQ))

The model prediction is as good as the PLSR model. Nonetheless, we expect this as the CNN model tends to do better in a larger dataset. The layers within the CNN model can be further optimized if needed. For the purpose of this tutorial, we assume that the model is fully optimized, and can be applied for the predictions of other properties measured.

In [ ]:
# Standardize the output
# This time, we can standardize X
# as we are going to predict all the properties using the CNN model.

sc_y = StandardScaler()
y_cal = sc_y.fit_transform(y_calX)
y_val = sc_y.transform(y_valX)

In [ ]:
n = test_spec.shape [0]
p = y_cal.shape[1]

# create empty matrix to save the predictions
Y_pred = np.zeros([n, p])

for x in range(0, p):
    # Create the 1D CNN model
    model = model1D(channel_number, nb_features, multOUT=False)

    # train the model
    h = model.fit(cal_spec, y_cal[:,x], validation_data=(val_spec,y_val[:,x]), epochs=EPOCH, batch_size = B_size)
    
#     Evaluate the model by plotting the mean absolute error of cal and val data
    plt.rcParams['figure.figsize'] = [20, 5]
    plt.subplot(1,4,x+1)
    
    plt.plot(h.history['mae'], label='mae')
    plt.plot(h.history['val_mae'], label = 'val_mae')
    plt.xlabel('Epoch')
    plt.ylabel('MAE')
    plt.title(properties[x], x=0.55, y= 0.9, fontsize ='small', loc='left')
    plt.legend(loc='upper right')
    
    # create predictions using trained model
    tmp = model.predict(test_spec)
    Y_pred[:,x] = np.reshape(tmp,-1)

In [ ]:
#inverse back the results
Y_predT = sc_y.inverse_transform(Y_pred)

plt.rcParams['figure.figsize'] = [20, 5]
for i in range(0, Y_predT.shape[1]):
    plt.subplot(1,4,i+1)
    plot_goof(y_test[:,i],Y_predT[:,i])
    plt.title(properties[i], fontsize ='small', loc='left')
    
#     print the accuracies for all the predictions
    r2, RMSE, bias, RPIQ ,_ = goof(y_test[:,i],Y_predT[:,i])    
    print('CNN model2', properties[i],'- R2: %0.3f, RMSE: %0.3f, bias: %0.3f, RPIQ:%0.3f' %(r2, RMSE, bias, RPIQ))

#### Multiple output predictions
The follwoing is an example where a single CNN model can predict multiple outputs, in this case 4 soil parameters at a time.

In [ ]:
# Create the 1D CNN model 
# Note that for this section, we set the multiple output (multOUT) predictions to be true.
model_mult = model1D(channel_number, nb_features, multOUT=True);

#Let's display the architecture of the model:
model_mult.summary()

In [ ]:
# train the model
h = model_mult.fit(cal_spec,[y_cal[:,0],y_cal[:,1],y_cal[:,2],y_cal[:,3]], validation_data=(val_spec,[y_val[:,0],y_val[:,1],y_val[:,2],y_val[:,3]]), epochs = EPOCH, batch_size = B_size)

In [ ]:
plt.rcdefaults()

# Evaluate the model by plotting the mean absolute error of cal and val data
plt.plot(h.history['val_Dense_OC.2_mae'], label = 'vOC_mae')
plt.plot(h.history['val_Dense_CEC.2_mae'], label = 'vCEC_mae')
plt.plot(h.history['val_Dense_clay.2_mae'], label = 'vclay_mae')
plt.plot(h.history['val_Dense_sand.2_mae'], label = 'vsand_mae')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend(loc='upper right')

In [ ]:
# create predictions using trained model
TT = model_mult.predict(test_spec)
Y_pred_mult = np.hstack(TT)

In [ ]:
#inverse back the results
Y_predT_mult = sc_y.inverse_transform(Y_pred_mult)

In [ ]:
plt.rcParams['figure.figsize'] = [20,5]

# plot the goodness of fit
for i in range(0, Y_predT_mult.shape[1]):
    plt.subplot(1,4,i+1)
    plot_goof(y_test[:,i],Y_predT_mult[:,i])
    plt.title(properties[i], fontsize ='small', loc='left')
    
#     print the accuracies for all the predictions
    r2, RMSE, bias, RPIQ ,_  = goof(y_test[:,i],Y_predT_mult[:,i])    
    print('CNN model_mult',properties[i],'- R2: %0.3f, RMSE: %0.3f, bias: %0.3f, RPIQ:%0.3f' %(r2, RMSE, bias, RPIQ))

### Visualize the feature maps in the convolution layers

The activation maps, called feature maps, capture the result of applying the filters to input. The idea behind it is to understand what spectra features are preserved to generate the predictions

In [ ]:
model_mult.summary()

In [ ]:
# Grab the index of the convolution layers
ixs=[]
for i in range(len(model_mult.layers)):
    layer = model_mult.layers[i]
    
    # check for convolutional layer
    if 'Conv' not in layer.name:
        continue
    ixs.append(i)
    
    # summarize output shape
    print(i, layer.name, layer.output.shape)

In [ ]:
test_data = test_spec[1:2,:]

for i in ixs:
#     get the output for each convolution layer
    outputs = model_mult.layers[i].output
    
#     create a new model to evaluate the output at a certain convolution layer
    feature_model = Model(inputs=model_mult.inputs, outputs=outputs)
               
    # get feature map for the convolution layer
    feature_maps = feature_model.predict(test_data)
    n_cols = 5
    
    for fmap in feature_maps:
        n_rows = fmap.shape[-1]//n_cols
        if i == ixs[0]:
            plt.rcParams['figure.figsize'] = [15,6]
        if i == ixs[1]:
            plt.rcParams['figure.figsize'] = [15,12]
        elif i == ixs[2]:
            plt.rcParams['figure.figsize'] = [15,12]
        fig = plt.figure()
        for num in range(fmap.shape[-1]):
            plt.subplot(n_rows, n_cols, num+1);
            plt.plot(fmap[:,num])
            plt.title('filter number #'+str((num+1)))
            plt.axis('off')

        fig.suptitle(model.layers[i].name+ str(': A few of the ')+str(fmap.shape[-1])+ str(' filters'),
                     x=fig.subplotpars.left, y=1.1, horizontalalignment='left', fontsize=20)        

### Variables of Importance

One of the shortcomings of CNN is that it is difficult to interpret the results from the neural network. One easy way to assess how the CNN model used spectral variables for the prediction of soil properties via sensitivity analysis. 

First, a new data frame was created by averaging all the calibration spectra data, and duplicated to the same number of samples as test data. The first five wavelengths of these new averaged spectra were replaced with the actual value from the test data. This simulated spectral data frame was fed into the trained CNN model, and the predictions of soil properties were given. The variance for the prediction of the four properties was then calculated as a measure of the importance of the spectral variables. This process was repeated until all the wavelengths had been evaluated as only five wavelengths were observed at any time.

If the wavelengths that were varied are important in predicting that particular properties, we expected the variance to be higher. Conversely, if the wavelengths that were changed were not important, then there should be low variance because the rest of the wavelengths’ variables are the averages of the test dataset. The sensitivity of spectral variables was unique for each soil property and could be related to the band assignments for particular wavelengths. Because the CNN model contained several shared layers, some of these selected spectral variables overlapped with different level of importance. 

In [ ]:
import random

### trying to find important variables used in prediction
nvar = 5 #change 5 variables/wavelength at a time
ntest = X_cal_MIR.shape[1]-nvar+1 # calculate number of tests

# average the calibration spectra
avemat = X_cal_MIR.mean(axis=0)
avemat = pd.DataFrame(avemat).T

# duplicate the average spectra to match the number of test data
avemat = avemat.append([avemat]*(len(X_test_MIR)-1),ignore_index=True)

In [ ]:
# create empty matrix to save the important variables
VarImp = np.zeros(shape=(ntest,y_cal.shape[1]))

for xx in range(0,ntest):
    
#     create a dummy matrix to get the average spectra matrix calculated previously
    dummymat = avemat
    
#     replace the dummy matrix with actual values from the test set
    dummymat.iloc[:,xx:xx+nvar] = X_test_MIR[:,xx:xx+nvar]

    ## reshape the dummy matrix into 3D matrix
    tempmat = np.expand_dims(dummymat,axis=2)
    
#     create the predictions
    TT = model_mult.predict(tempmat)
    Y_pred_new = np.hstack(TT)
    Y_pred_newT = sc_y.inverse_transform(Y_pred_new)
    
#    determine the variance of the predictions
    tmp = Y_pred_newT.var(axis=0)
    
#     save the results
    VarImp[xx] = tmp

In [ ]:
VarImp = pd.DataFrame(VarImp.T)

wave_imp = wavet[2:-2]
# rename the columns using the wavelengths
VarImp.columns = wave_imp

In [ ]:
#Let's plot to observe which wavelengths were utilized within the model for the prediction of various properties
for i in range(VarImp.shape[0]):
    plt.rcParams['figure.figsize'] = [10,12]
    plt.subplot(4,1,i+1)
    plt.plot(wave_imp, VarImp.iloc[i,:].T)
    plt.title(properties[i], x=0.95, y= .8, fontsize ='small', loc='left')
    
    ax = plt.gca()
    ax.invert_xaxis()
    ax.axes.yaxis.set_visible(False)

plt.show()

In [ ]:
end = time.time()
print('Time elapsed:', end - start)